In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import ast
from tqdm import tqdm
import time
from collections import Counter
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [2]:
data=pd.read_csv('model data/final.csv')
data.shape

(142776, 14)

In [3]:
data['population']=data['population'].astype('float64')

In [4]:
ind=list(data[data['topography_elevation']==99999].index)
for i in ind:
    data.at[i,'topography_elevation']=0

In [5]:
data

,latitude,longitude,nkill,days_passed,DIS_LAKE,DIS_MAJOR_RIVER,DIS_OCEAN,PRECI_AVG,TEMP_AVG,population,ethnic_groups,topography_elevation,nightlight_intensity,happy_index
0,18.5,-70.0,1,17834,1415040.0,1285293.0,33082.41,1703.198276,26.640805,2180.63,99999,62.00,59.82,4.885
1,34.8,37.3,1,1406,73722.0,1133031.0,140738.70,198.627586,17.571624,14.11,1,821.48,0.94,3.006
2,14.5,44.4,1,1407,826500.5,1737351.0,196373.80,347.809479,20.158693,214.15,3,2445.19,10.57,4.077
3,36.5,40.7,1,1407,167311.2,943071.2,432522.10,542.012062,16.618319,178.64,3,311.09,10.94,3.006
4,33.6,36.3,1,1406,199295.5,1169763.0,56242.69,516.005172,14.889152,74.82,1,1157.07,31.62,3.006
5,33.0,70.3,1,1407,661686.7,2045339.0,956084.30,534.413793,16.999354,138.59,1,634.88,3.07,5.194
6,33.8,44.6,1,1380,178369.6,524083.4,553956.80,374.599141,22.104741,60.19,2,43.75,34.03,4.677
7,15.3,44.2,1,1406,826500.5,1737351.0,196373.80,347.809479,20.158693,178.64,3,2322.18,55.11,99999.000
8,35.9,10.6,1,1404,287681.5,1262467.0,21061.60,453.281600,18.368390,99999.00,99999,0.00,44.96,4.739
9,32.2,35.5,1,1404,429615.3,1241544.0,26881.02,445.222414,19.475215,154.52,3,0.00,10.16,7.278


In [6]:
df = data.replace(99999, pd.np.nan)

In [7]:
data=df.dropna()

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117511 entries, 1 to 142775
Data columns (total 14 columns):
latitude                117511 non-null float64
longitude               117511 non-null float64
nkill                   117511 non-null int64
days_passed             117511 non-null int64
DIS_LAKE                117511 non-null float64
DIS_MAJOR_RIVER         117511 non-null float64
DIS_OCEAN               117511 non-null float64
PRECI_AVG               117511 non-null float64
TEMP_AVG                117511 non-null float64
population              117511 non-null float64
ethnic_groups           117511 non-null float64
topography_elevation    117511 non-null float64
nightlight_intensity    117511 non-null float64
happy_index             117511 non-null float64
dtypes: float64(12), int64(2)
memory usage: 13.4 MB


In [9]:
data['nkill'].value_counts()

1    59541
0    57970
Name: nkill, dtype: int64

In [95]:
data = data.sample(frac=1).reset_index(drop=True)
X2=data.drop(columns=['days_passed'])
#X=X1.groupby(['latitude','longitude','DIS_LAKE','DIS_MAJOR_RIVER','DIS_OCEAN','TEMP_AVG','PRECI_AVG','population','ethnic_groups','topography_elevation','nightlight_intensity','happy_index']).size().reset_index(name="frequency")
X1=X2.groupby(['latitude','longitude','DIS_LAKE','DIS_MAJOR_RIVER','DIS_OCEAN','TEMP_AVG','PRECI_AVG','population','ethnic_groups','topography_elevation','nightlight_intensity','happy_index']).max().reset_index()
#X.head()
X=X1.drop(columns=['nkill'])

y=X['nkill']
#X=X.drop(columns=['nkill'])


KeyError: 'nkill'

In [79]:
#X=X.reset_index()
X.head()
X.shape

(24614, 13)

In [80]:
scaler = StandardScaler()
print(scaler.fit(X))

StandardScaler(copy=True, with_mean=True, with_std=True)


In [81]:
X=scaler.transform(X)

In [82]:
from sklearn import model_selection
X_train, X_val, y_train, y_val = model_selection.train_test_split(X, y, test_size=0.20, random_state=101)

In [83]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

In [84]:
logClf = LogisticRegression(solver='lbfgs', multi_class='multinomial',random_state=1)
rfClf = RandomForestClassifier(n_estimators=50, random_state=1)
gaussClf = GaussianNB()

In [85]:
eclf = VotingClassifier(estimators=[('lr', logClf), ('rf', rfClf), ('gnb', gaussClf)], voting='hard')

In [86]:
eclf.fit(X_train,y_train)


VotingClassifier(estimators=[('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=1, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)), ('rf', Random...   oob_score=False, random_state=1, verbose=0, warm_start=False)), ('gnb', GaussianNB(priors=None))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [87]:
from sklearn.model_selection import cross_val_score

In [88]:
for clf, label in zip([logClf, rfClf, gaussClf, eclf], ['Logistic Regression', 'Random Forest', 'naive Bayes', 'Ensemble']):
    scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.55 (+/- 0.01) [Logistic Regression]
Accuracy: 0.44 (+/- 0.00) [Random Forest]
Accuracy: 0.55 (+/- 0.00) [naive Bayes]


C:\vidushi\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\vidushi\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\vidushi\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\vidushi\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

Accuracy: 0.53 (+/- 0.00) [Ensemble]
